# Runtime benchmarks

This notebook is designed to calculate benchmark runtimes on the ORIO website.

Note that benchmarks are highly dependent on the number of processors available; the feature-list count-matrixes are parallelized using a task manager, and are also cached so the don't need to be recalculated in the future.


## User inputs, modify environment:

Set environment variables as needed before running:

```bash
export "ORIO_BENCHMARK_EMAIL=foo@bar.com"
export "ORIO_BENCHMARK_FEATURELIST=/path/to/hg19_fake.filtered.bed"
export "ORIO_BENCHMARK_OUTPUT=/path/to/benchmark_output.txt"
```

## Startup

In [ ]:
%matplotlib inline

from collections import namedtuple
from io import BytesIO
import os 
import time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from scipy import stats

from django.core.files import File 

from analysis import models
from myuser.models import User

pd.options.display.mpl_style = 'default'  # ggplot

In [ ]:
# setup user inputs
email = os.environ['ORIO_BENCHMARK_EMAIL']
bigFeatureList = os.environ['ORIO_BENCHMARK_FEATURELIST']
outputs = os.environ['ORIO_BENCHMARK_OUTPUT']
replicates = 3
featureNs = [10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000]
datasetNs = [2, 5, 10, 50, 100, 500, 750]


featureNs = [10, 50]
datasetNs = [2, 5]

## Clear old benchmark results

In [ ]:
user = User.objects.get(email=email)
models.FeatureList.objects\
    .filter(owner=user, name__icontains='benchmarking:')\
    .delete()

## Create feature lists

We take a list of over 130,000 features, and then randomly select a subset of features from this master set. Then, we create a list of FeatureLists, each with a different number of features.

In [ ]:
# load big feature-list file
with open(bigFeatureList, 'r') as f:
    fls = f.readlines()

fls = np.array(fls)
print('{:,} lines'.format(fls.size))
print('First line: %s ' % fls[0])
print('Last line: %s' % fls[-1])

In [ ]:
def getFeatureList(features, size):
    fl = features[np.random.choice(features.size, size, replace=False)]
    f  = BytesIO()
    bytestring = str.encode(''.join(fl.tolist()))
    f.write(bytestring)
    f.seek(0)
    return f

In [ ]:
# create feature-list objects in Django
featureLists = []
for n in featureNs:
    name = "benchmarking: {} features".format(n)
    fl = models.FeatureList.objects.create(
        owner=user,
        name=name,
        stranded=True,
        genome_assembly_id=1,  # hg19
    )    
    fl.dataset.save(name+'.txt', File(getFeatureList(fls, n)))
    fl.save()
    fl.validate_and_save()
    featureLists.append((n, fl))

In [ ]:
# delete existing feature-list count matrices; required becase
# it will change the benchmarking behavavior because by 
# default the matrix can be re-used after initial exeuction.
def deleteFlcm():
    fls = [fl[1] for fl in featureLists]
    models.FeatureListCountMatrix.objects\
        .filter(feature_list__in=fls)\
        .delete()

## Generate random dataset collections

We randomly select a subset of encode datasets of varying sizes. To try to make the datasets a little more uniform for benchmarking, we first select the largest subset, and then iteratively select smaller subsets from each previous subset (that way we know that the smallest subset is guarenteed to a set of datasets which were previously run in a larger dataset.

The end result is a list of datasets, going from smallest to largest.

In [ ]:
# get available datasets
datasetLists = []
datasets = list(models.EncodeDataset.objects\
    .filter(genome_assembly_id=1)\
    .values_list('id', 'name'))

# create subsets
for n in reversed(datasetNs):
    subset_ids = np.random.choice(len(datasets), n, replace=False)
    subset = [datasets[i] for i in subset_ids]
    datasetLists.append([dict(dataset=d[0], display_name=d[1]) for d in subset])
    datasets = subset
    
# switch order to go from smallest -> largest
datasetLists = list(reversed(datasetLists))

## Create analyses

We create and validate our analyses, where there will be a total of $i * j * k$, where $i$ is the number of feature lists, $j$ is the number of dataset lists, and $k$ is the number of replicates for each.

In [ ]:
# create analyses
analyses = []
for fl in featureLists:
    n_features = fl[0]
    for ds in datasetLists:
        for rep in range(replicates):
            n_ds = len(ds)

            a = models.Analysis.objects.create(
                owner=user,
                name="benchmarking: {} features, {} datasets".format(n_features, n_ds),
                genome_assembly_id=1,  # hg19
                feature_list=fl[1],
            )
            a.save()        
            objects = [
                models.AnalysisDatasets(
                    analysis_id=a.id,
                    dataset_id=d['dataset'],
                    display_name=d['display_name'],
                ) for d in ds
            ]
            models.AnalysisDatasets.objects.bulk_create(objects)
            a.validate_and_save()
            analyses.append((a, n_features, n_ds))

## Execution

Now, we iteratively execute each analysis. We don't start the next analysis until the previous has finished.

Results are saved, and then transformed into a pandas DataFrame, and exported.

In [ ]:
# execute
Analysis = namedtuple('Analysis', ('id', 'features', 'datasets', 'seconds'))
results = []
for i, analysis in enumerate(analyses):
    print('Running {} of {}...'.format(i+1, len(analyses)))
    deleteFlcm()
    analysis[0].execute(silent=True)
    while True:
        time.sleep(3)
        a = models.Analysis.objects.get(id=analysis[0].id)
        if a.is_complete:
            break
    
    duration = (a.end_time-a.start_time).total_seconds()
    results.append(Analysis(a.id, analysis[1], analysis[2], duration))            

In [ ]:
# save and export
res = pd.DataFrame(results)
res.head(10)
res.to_csv(outputs, sep='\t',index=False)

## Visuals

Create a lineplot and regression for datasets vs runtime.

In [ ]:
subset = res[res.features==10]
f = subset.plot(kind='scatter', x='datasets', y='seconds', figsize=(12, 8))
f.hold= True
m = stats.linregress(x=subset.datasets.values, y=subset.seconds.values)
xs = np.arange(0, subset.datasets.max()*1.2, 1)
ys = m.intercept + m.slope * xs
f.plot(xs, ys, 'r-')
print(m)

Create a scatterplot for features vs runtime.

In [ ]:
subset = res[res.datasets==10]
plt = subset.plot(kind='scatter', x='features', y='seconds', figsize=(12, 8))
plt.hold= True
m, residuals, rank, singular_values, rcond = np.polyfit(subset.features.values, subset.seconds.values, 2, full=True)
m = list(reversed(m))
xs = np.arange(0, subset.features.max()*1.2, 5)
ys = m[0] + m[1] * xs + m[2] * np.power(xs, 2)
plt.plot(xs, ys, 'r-')

Create a contour plot, where x = features, y = datasets, and contour is runtime.

In [ ]:
import matplotlib.pyplot as plt
plt.set_cmap(cm.gist_earth)

res2 = res.drop('id', axis=1)
res2 = res2.groupby(['features', 'datasets']).mean()
res2.head()

X=res2.index.levels[0].values
Y=res2.index.levels[1].values
Z=res2.unstack(level=0).seconds.values

plt.figure(figsize=(12, 8))
ct = plt.contour(X, Y, Z)

plt.clabel(ct, inline=1, fmt='%d', fontsize=12)
plt.xlabel('N features')
plt.ylabel('N datasets')
plt.title('Total runtime for ORIO analysis')

plt.colorbar(ct, orientation='horizontal', shrink=0.8)